In [16]:
from bifrost.fdmt import Fdmt
import numpy as np
import pylab as plt
import bifrost as bf

from astropy import units as u
import setigen as stg
import matplotlib.pyplot as plt

import hyperseti
import cupy as cp
import blimpy as bl

fdmt = Fdmt()

In [17]:
import blimpy as bl

fp = 'guppi_59334_59899_6179049297_ProxCen_S_0001.0002.fil'
fil = bl.Waterfall(fp)

In [19]:
import numpy as np

max_delay = 128
gulp_size = 4 * 8192

def get_gulp(idx):
    d_cpu = np.ascontiguousarray(np.expand_dims(fil.data.squeeze().T, axis=0)[..., idx * gulp_size:(idx+1) * gulp_size])
    return d_cpu

d_cpu = get_gulp(0)

## Initialize FDMT
n_disp = max_delay
n_time = d_cpu.shape[2]
n_chan = d_cpu.shape[1]
fdmt.init(n_chan, n_disp, fil.header['fch1'], fil.header['foff'])

print(d_cpu.shape)

# Input shape is (1, n_freq, n_time)
d_in = bf.ndarray(d_cpu, dtype='f32', space='cuda')
d_out = bf.ndarray(np.zeros(shape=(1, n_disp, n_time)), dtype='f32', space='cuda')

print(d_in.shape, d_out.shape)
print(n_chan, n_time, n_disp, n_time)

# Execute FDMT
fdmt.execute(d_in, d_out, negative_delays=True)

d_out = d_out.copy(space='system')

(1, 64000, 4, 668)
(1, 64000, 4, 668) (1, 128, 4)
64000 4 128 4


RuntimeError: b'BF_STATUS_INVALID_SHAPE'